In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np

In [2]:
import sys
sys.getdefaultencoding()

'utf-8'

Open the questionnaire file (word document version of the questionnaire saved as html):

In [3]:
questionnaire_file = open("FF_m1_Questionnaire_2013.htm", "r") 

In [4]:
soup=BeautifulSoup(questionnaire_file, "html.parser")

In [5]:
ps=[p for p in soup.find_all('p')]

In [6]:
ps[164:168]

[<p class="MsoNormal" style="margin-left:1.0in;line-height:150%;text-autospace:
 none"><span style='font-size:11.0pt;line-height:150%;font-family:"Arial",sans-serif;
 color:black'>YES, MARRIED TO MOTHER..................... 1 <b>(SKIP TO B22 ON
 P.14)</b></span></p>,
 <p class="MsoNormal" style="margin-left:1.0in;line-height:150%;text-autospace:
 none"><span style='font-size:11.0pt;line-height:150%;font-family:"Arial",sans-serif;
 color:black'>NO, NOT MARRIED TO MOTHER.............. 2 </span></p>,
 <p class="MsoNormal" style="line-height:150%;text-autospace:none"><span style='font-size:11.0pt;line-height:150%;font-family:"Arial",sans-serif;
 color:black'> </span></p>,
 <p class="MsoNormal" style="text-autospace:none"><span style="position:relative;
 z-index:251606016"><span style="position:absolute;left:-1px;top:-1px;
 width:566px;height:3px"><img height="3" src="FF_f1_Questionnaire_2013_files/image007.gif" width="566"/></span></span><b><span style='font-size:11.0pt;font-family:"Arial"

Opening the excel file with all the questions for p6, using the pandas python library:

In [7]:
df=pd.read_excel("FFCm1.xlsx") #this is an excel file with all the questions for p6

In [8]:
df.head()

,position,name,varlab,probe
0,435,mothid1,Encrypted mother's ID,NaN
1,436,m1intmon,Mother interview month,NaN
2,437,m1intyr,Mother interview year,NaN
3,438,m1lenhr,What was the total length of interview - Hours,NaN
4,439,m1lenmin,What was the total length of interview - Minutes,NaN


In [9]:
df.shape

(331, 4)

In [10]:
reg = re.compile(r'(?P<section>[A-Za-z]*\d*)(?P<rest>.*)')

def format_q_num(name):
    """takes in the question numbers in the name column and outputs them in the format
    they appear in the text"""
    name=name.lstrip('m1')
    name=reg.search(name)
    section=name.group('section').upper()
    rest=name.group('rest').upper()
    return section+rest

In [11]:
df['question_number']=df.name.apply(lambda x: format_q_num(x))

In [12]:
df['multiple_answers']=df.question_number.apply(lambda x: 1 if re.match('[A-Z]\d*[A-Z]\d*',x) else 0) 
#adds a column where 1 indicates that it is one of the questions with multiple entries in the data frame
#I think these are mostly (only?) the "select all that apply" questions

In [13]:
df[66:75]

,position,name,varlab,probe,question_number,multiple_answers
66,501,m1b9,Do you and BF have plans to live together or m...,NaN,B9,0
67,502,m1b9a1,"Reasons no plan to live tgthr, No need to marry?",NaN,B9A1,1
68,503,m1b9a2,"Reasons no plan to live tgthr, Timing problem ?",NaN,B9A2,1
69,504,m1b9a3,"Reason no plan to live tgthr, Financial Reasons ?",NaN,B9A3,1
70,505,m1b9a4,"Reasons no plan to live tgthr, Distance ?",NaN,B9A4,1
71,506,m1b9a5,"Reasons no plan to live tgthr, Incarceration ?",NaN,B9A5,1
72,507,m1b9a6,"Reasons no plan to live tgthr, Housing reasons ?",NaN,B9A6,1
73,508,m1b9a7,"Reason no plan to live tgthr, Relationship re...",NaN,B9A7,1
74,509,m1b9a8,"Reasons no plan to live tgthr, Drug problem ?",NaN,B9A8,1


In [14]:
def find_question(q_number):
    """finds where the question number appears in the text - only if it only appears once"""
    ps_text = []
    ps_idx = []
    ps_startswith = []
    for idx, p in enumerate(ps):
        if re.match('[A-Z]\d*[A-Z]',q_number):
            if q_number in p.get_text():
                if p.get_text().lstrip().startswith(q_number+'.'):
                    ps_text.append(p.get_text().lstrip().split('.')[1])
                    ps_idx.append(idx)
                else:
                    ps_text.append(p.get_text().lstrip().split('.')[0])
                    ps_idx.append(idx)
        else:
            if q_number+"." in p.get_text():
                if p.get_text().lstrip().startswith(q_number+"."):
                    ps_text.append(p.get_text().lstrip().split('.')[1])
                    ps_idx.append(idx)
    if len(ps_idx)!=1:
        return np.nan, np.nan
    else:
        return ps_text[0].replace("\n"," ").strip(), ps_idx[0]

In [15]:
s = (find_question('B5'))
s

('I’m going to read you some things that couples often do together', 219)

In [16]:
def find_probe(q_number):
    probe = re.findall('[A-Z]{,1}\d{,2}', q_number)[0]
    return find_question(probe)[0]

In [17]:
find_probe('A3')

'Have you held the [baby/babies]?'

In [18]:
df.drop(['probe'],axis=1,inplace=True)

In [19]:
df = df.drop(df.index[0:16])
df.index = range(len(df))

In [20]:
df

,position,name,varlab,question_number,multiple_answers
0,451,m1a3,Have you picked up a (name/names) for the (bab...,A3,0
1,452,m1a4,Is respondent married to baby's father?,A4,0
2,453,m1a5,Will the baby (babies) have the father's last ...,A5,0
3,454,m1a6,Will the father's name be on the birth certifi...,A6,0
4,455,m1a7,Has BF visited you in the hospital?,A7,0
5,456,m1a8,When are you going home?,A8,0
6,457,m1a9,Do you feel ready to go home or would you rath...,A9,0
7,458,m1a10,Int chk: Is respondent married to (BF)?,A10,0
8,459,m1a11a,Will the baby live with mother?,A11A,1
9,460,m1a11b,Will the baby live with father?,A11B,1


In [21]:
for i in range(len(df.question_number)):
    q_number = df.loc[i,'question_number']
    if re.match('^[A-Z]{,1}\d{,2}', q_number):
        df.loc[i,'probe'] = find_probe(q_number)
    else:
        df.loc[i,'probe'] = "None"

In [22]:
df

,position,name,varlab,question_number,multiple_answers,probe
0,451,m1a3,Have you picked up a (name/names) for the (bab...,A3,0,Have you held the [baby/babies]?
1,452,m1a4,Is respondent married to baby's father?,A4,0,Will the (baby/babies) have your last name?
2,453,m1a5,Will the baby (babies) have the father's last ...,A5,0,Will your name be on the birth certificate?
3,454,m1a6,Will the father's name be on the birth certifi...,A6,0,Do you have any other biological children?
4,455,m1a7,Has BF visited you in the hospital?,A7,0,Who (does/do) the (baby/babies) look like?
5,456,m1a8,When are you going home?,A8,0,NaN
6,457,m1a9,Do you feel ready to go home or would you rath...,A9,0,NaN
7,458,m1a10,Int chk: Is respondent married to (BF)?,A10,0,NaN
8,459,m1a11a,Will the baby live with mother?,A11A,1,NaN
9,460,m1a11b,Will the baby live with father?,A11B,1,NaN


In [23]:
df['ps_text'], df['ps_idx'] = zip(*df['question_number'].map(find_question))

In [24]:
df[24:29]

,position,name,varlab,question_number,multiple_answers,probe,ps_text,ps_idx
24,475,m1b1b,How many months did you know Baby's Father bef...,B1B,1,NaN,B1A WRITE IN YEARS oo ...,162.0
25,476,m1b2,Int chk: Is respondent married (OFFICIAL MARRI...,B2,0,NaN,NaN,NaN
26,477,m1b3,Which statement best describes your current re...,B3,0,Which of the following statements best describ...,Which of the following statements best describ...,168.0
27,478,m1b4a,"Why did rom rel end with (BF), Financial Reaso...",B4A,1,Why did your romantic relationship with [BABY’...,"B4A FINANCIAL REASONS (DON’T HAVE WORK, M...",205.0
28,479,m1b4b,"Why did rom rel end with (BF), Distance (Don't...",B4B,1,Why did your romantic relationship with [BABY’...,B4B DISTANCE (DON’T LIVE IN SAME TOWN),206.0


In [25]:
np.isnan(df.ps_idx).sum() #number of questions where the question number doesn't appear or appears more than once

121

In [26]:
print(df.ps_text[16])
print(df.ps_idx[16])

nan
nan


In [27]:
#get index of <p> tag for the next question so we can split the questions (can't just shift because some questions have more than one answer or missing values)
all_idx=np.array(df.ps_idx)
all_idx=all_idx[~np.isnan(all_idx)]
all_idx=np.append(all_idx, [len(ps)])
df['ps_idx_next']=df.ps_idx.apply(lambda x: all_idx[all_idx > x].min() if ~np.isnan(x) else np.nan)

In [28]:
df[42:43]

,position,name,varlab,question_number,multiple_answers,probe,ps_text,ps_idx,ps_idx_next
42,493,m1b6e,When you and BF tgthr how often did you disagr...,B6E,1,The following is a list of subjects on which c...,Drinking or drug use ?,246.0,247.0


In [29]:
a3=ps[105:110]
a3

[<p class="MsoNormal" style="line-height:150%;text-autospace:none"><b><span style='font-size:11.0pt;line-height:150%;font-family:"Arial",sans-serif;
 color:black'>A2.</span></b><span style='font-size:11.0pt;line-height:150%;
 font-family:"Arial",sans-serif;color:black'> Were you present at the birth?</span></p>,
 <p class="MsoNormal" style="margin-left:1.0in;line-height:150%;text-autospace:
 none"><span style='font-size:11.0pt;line-height:150%;font-family:"Arial",sans-serif;
 color:black'>YES................................................................ 1</span></p>,
 <p class="MsoNormal" style="margin-left:1.0in;line-height:150%;text-autospace:
 none"><span style='font-size:11.0pt;line-height:150%;font-family:"Arial",sans-serif;
 color:black'>NO................................................................. 2</span></p>,
 <p class="MsoNormal" style="text-indent:.5in;line-height:150%;text-autospace:
 none"><span style="position:relative;z-index:251601920;left:-1px;top:11px;
 width:

In [30]:
consequitivedots = re.compile(r'\.{3,}')

In [31]:
combined = [(p.find_previous('p').get_text() + p.get_text()) for p in a3]


In [32]:
text = ''
j = 0
for p in a3:
    print(j)
    print(p.get_text())
    text = text + ' ' + p.find_previous('p').get_text()
    j += 1

0
A2. Were you present at the birth?
1
YES................................................................ 1
2
NO................................................................. 2
3
 
4
A3. Have you held the [baby/babies]?


In [33]:
print(text)

   A2. Were you present at the birth? YES................................................................ 1 NO................................................................. 2  


test if lowercase letter, if so grab previous p as well.

In [34]:
answers = [p for p in a3 if len(p.find_all(string = consequitivedots))>0]

In [35]:
type(answers[0])

bs4.element.Tag

In [36]:
print(answers[0].prettify())

<p class="MsoNormal" style="margin-left:1.0in;line-height:150%;text-autospace:
none">
 <span style='font-size:11.0pt;line-height:150%;font-family:"Arial",sans-serif;
color:black'>
  YES................................................................ 1
 </span>
</p>



In [37]:
test_answers = [p for p in a3 if len(p.find_all(string = consequitivedots))>0]
for ans in test_answers:
    if re.match('^[a-z]{1}', ans.get_text().split('.')[0]):
        print(ans.find_previous('p').get_text() + ans.get_text().split('.')[0])
        print(ans.get_text().split("(")[0].split('.')[-1])
    else:
        print(ans.get_text().split('.')[0])
        print(ans.get_text().split('.')[-1])

YES
 1
NO
 2


In [38]:
def clean_text(string):
    return string.replace("\n"," ").strip()

def get_pairs(p):
    if re.match('^[a-z]{1}', p.get_text().split('..')[0]):
        if re.match('[()]{1}', p.get_text().split('..')[-1]):
            pairs = [p.find_previous('p').get_text() + ' ' + p.get_text().split('..')[0], p.get_text().split("(")[0].split('..')[-1]]
        else:
            pairs = [p.find_previous('p').get_text() + ' ' + p.get_text().split('..')[0], p.get_text().split('..')[-1]]
    else:
        if re.match('[()]{1}', p.get_text().split('.')[-1]):
            pairs = [p.get_text().split('..')[0], p.get_text().split("(")[0].split('..')[-1] ]
        else:
            pairs = [p.get_text().split('..')[0], p.get_text().split('..')[-1]]
    pairs = [clean_text(text) for text in pairs]
    return pairs
        
def get_answers(start_idx,stop_idx):
    if np.isnan(start_idx) or np.isnan(stop_idx):
        return ([],0)
    else:
        answers = [p for p in ps[int(start_idx):int(stop_idx)] if len(p.find_all(string = consequitivedots))>0]
        answers = [get_pairs(a) for a in answers]
        return(answers, len(answers)) 
        

In [39]:
df['answers'], df['n_answers'] = zip(*map(get_answers, df['ps_idx'], df['ps_idx_next']))

In [40]:
(df['answers'][16])

[]

In [41]:
x = 0
while x < len(df.answers):
    ans = df.answers[x]
    if len(ans) > 0:
        for y in range(len(ans)):
            df.loc[x,'val{0}'.format(y)] = ans[y][1] #create new 'val(y)' column
            df.loc[x,'lab{0}'.format(y)] = ans[y][0] #create new 'lab(y)' column
    x += 1

In [42]:
def clean_answers(col):
    col.str.replace('.','',1)

In [43]:
values = []
for i in range(0,18):
    values = values + ['val{0}'.format(i)]
    i += 1
for v in values:
    df[v] = df[v].map(lambda x: str(x))
    df[v] = df[v].map(lambda x: x.lstrip('.').strip())

In [44]:
df['val0']

0                            1
1                            1
2               1 (SKIP TO A6)
3                1 (GO TO A6A)
4                            1
5                          nan
6                          nan
7                          nan
8                          nan
9                          nan
10                         nan
11                         nan
12                         nan
13                         nan
14                         nan
15                         nan
16                         nan
17                         nan
18                         nan
19                         nan
20                         nan
21                         nan
22                         nan
23     1 (SKIP TO B22 ON P.14)
24     1 (SKIP TO B22 ON P.14)
25                         nan
26      1 (SKIP TO B8 ON P. 6)
27                           1
28                           2
29                           3
                ...           
285                        nan
286     

In [45]:
i = 0
while i < len(df.ps_text):
    if df.multiple_answers[i] == 1:
        df.loc[i,'val0'] = '1'
        df.loc[i,'lab0'] = 'YES'
        df.loc[i,'val1'] = '2'
        df.loc[i,'lab1'] = 'NO'
    i += 1

In [46]:
df.drop(['answers','n_answers','question_number','ps_idx','ps_idx_next'],axis=1,inplace=True)

In [47]:
df.head(20)

,position,name,varlab,multiple_answers,probe,ps_text,val0,lab0,val1,lab1,...,val78,lab78,val79,lab79,val80,lab80,val81,lab81,val82,lab82
0,451,m1a3,Have you picked up a (name/names) for the (bab...,0,Have you held the [baby/babies]?,Have you held the [baby/babies]?,1,YES,2,NO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,452,m1a4,Is respondent married to baby's father?,0,Will the (baby/babies) have your last name?,Will the (baby/babies) have your last name?,1,YES,2,NO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,453,m1a5,Will the baby (babies) have the father's last ...,0,Will your name be on the birth certificate?,Will your name be on the birth certificate?,1 (SKIP TO A6),YES,2 (GO TO A5A),NO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,454,m1a6,Will the father's name be on the birth certifi...,0,Do you have any other biological children?,Do you have any other biological children?,1 (GO TO A6A),YES,2 (SKIP TO A7),NO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,455,m1a7,Has BF visited you in the hospital?,0,Who (does/do) the (baby/babies) look like?,Who (does/do) the (baby/babies) look like?,1,ME,2,BABY’S MOTHER,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,456,m1a8,When are you going home?,0,NaN,NaN,nan,NaN,nan,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,457,m1a9,Do you feel ready to go home or would you rath...,0,NaN,NaN,nan,NaN,nan,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,458,m1a10,Int chk: Is respondent married to (BF)?,0,NaN,NaN,nan,NaN,nan,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,459,m1a11a,Will the baby live with mother?,1,NaN,NaN,1,YES,2,NO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,460,m1a11b,Will the baby live with father?,1,NaN,NaN,1,YES,2,NO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
df.to_csv(path_or_buf = "m1Qtext.csv")